# Primer scrapeo

El proceso de scrapear una página web es parecido a lo que hace un humano cuando quiere buscar algo en Internet, la diferencia es que en lugar de ver el contenido presentado por el navegador, el programa analiza y selecciona el código fuente generalmente programado en  HTML y JavaScript.

El primer paso es por tanto seleccionar una página web para hacer el scraping y descargarla. Ya sabemos cómo hacer peticiones HTTP mediante `requests` así que vamos a a hacer una petición de la web de ejemplo por excelencia:

In [ ]:
import requests

req = requests.get("https://example.com")

El caso es que como respuesta a la petición se nos ha develto la página y podemos ver su código fuente en crudo:

In [ ]:
print(req.text)

Aquí tenemos un documento HTML bien estructurado con sus etiquetas.

Cuando el navegador interpreta estas etiquetas que se abren y se cierran, con sus atributos y contenidos genera lo que se conoce como **DOM** (*Document Object Model*), una interfaz de programación para documentos HTML y XML que en esencia es como un árbol ramificado de  componentes padres e hijos. El padre de todo es `html`, que tiene dos hijos `head` y `body`, el primero contiene el `title` y los metadatos, el otro el contenido de la página, una capa `div` que a su vez tiene una cabecera `h1` y unos parágrados `p`.

Pues bien, la biblioteca `BeautifulSoup` lo que hace es generar su propia estructura parecida a la interfaz **DOM** pero en Python, creando un árbol con los elementos del documento. Básicamente le pasamos un documento HTML en crudo y ella lo transforma en un objeto dinámico con el que podemos interactuar:

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text)

print(soup)

A simple vista parece lo mismo, pero ahora podemos hacer algo como esto para consultar el título de la página:

In [ ]:
soup.select("title")

Esto que nos devuelve es un objeto, veamos tu tipo:

In [ ]:
type(soup.select("title"))

Es un conjunto de resultados que contiene los tags que concuerdan con el nombre `title`, por tanto es una lista.

Veamos qué tipo tiene ese primer valor del conjunto:

In [ ]:
type(soup.select("title")[0])

Como véis es un `Tag` y éste contiene diferentes métodos, como por ejemplo `getText()` para recuperar su contenido:

In [ ]:
soup.select("title")[0].getText()

Podemos recuperar otros elementos esenciales como la cabecera o los parágrafos:

In [ ]:
soup.select("h1")

In [ ]:
soup.select("p")

Fijaros que el segundo parágrafo contiene a su vez un enlace, podemos acceder de forma anidada:

In [ ]:
# Seleccionar del segundo parágrafo el primer enlace 
a = soup.select("p")[1].select("a")[0]

# Mostrar su contenido
a.getText()

Las etiquetas tienen valores especiales llamados atributos, como la dirección `href` de un enlace. 

Estos se almacenan como un diccionario del objeto, es muy cómodo acceder a ellos:

In [ ]:
# Atributo con la dirección del enlace
a['href']

Estos valores están mapeados del diccionario `attrs`:

In [ ]:
a.attrs.items()

Siguiendo esta lógica podemos programar un script que recupere todos los atributos de los metadatos:

In [ ]:
for meta in soup.select("meta"):
    for atributo, valor in meta.attrs.items():
        print(f"{atributo}: {valor}")

Solo con esto os podéis hacer una ideal del potencial que tiene.